# Описание проекта

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### План работы

[Bert](#b)
1. [Подготовка данных](#b_1)
2. [Обучение модели](#b_2)

[tf-idf](#t)

1. [Подготовка данных](#t_1)
2. [Обучение модели](#t_2)

[Выводы](#c)

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

### Импорты библиотек

In [1]:
import os
from urllib.request import urlretrieve

import pandas as pd
import numpy as np
from tqdm import tqdm

# NLP
import torch
import transformers
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# ML
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, SGDClassifier
#from catboost import CatBoostClassifier

<a id='1'></a>
# 1. Подготовка

In [2]:
def get_file(file_name, url):
    if not os.path.exists(file_name):
        print('Файл не найден и будет загружен из сети')
        file_name, headers = urlretrieve(url)
    return pd.read_csv(file_name)

In [3]:
file_name = 'toxic_comments.csv'
url = 'datasets/toxic_comments.csv'

df = get_file(file_name, url)
print('Размер датасета:', df.shape)
df.head()

Файл не найден и будет загружен из сети
Размер датасета: (159571, 2)


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<a id='b'></a>
# BERT

<a id='b_1'></a>
### 1. Подготовка

Сразу выделим тестовую выборку и сбалансируем трейн с помощью downsampling

In [4]:
train, test = train_test_split(df, test_size=0.2, random_state=333)

train_0 = train[train['toxic'] == 0].sample(10000, random_state=333)
train_1 = train[train['toxic'] == 1].sample(10000, random_state=333)
train = shuffle(pd.concat([train_0]+[train_1]))

test = test.sample(10000, random_state=333)

print('Размер трейна', train.shape)
print('Размер теста', test.shape)

Размер трейна (20000, 2)
Размер теста (10000, 2)


2. Выполняем токенизацию каждого текста, то есть разбиваем на слова 
(Алгоритм лемматизации и очистки текста уже заложен в модели Bert)
4. На выходе у каждого исходного текста образуется свой список токенов.

In [5]:
%%time

# tockenize
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')
tokenized = train['text'].apply(
    lambda x: tokenizer.encode(x[:512], add_special_tokens=True))

# count max qty of tockens
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# add 0 to strokes which lenght less than max
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
# make a mask to highlight important tockens
attention_mask = np.where(padded != 0, 1, 0)
padded.shape

Wall time: 14.5 s


(20000, 486)

7. Затем токены передаем модели, которая переводит их в векторные представления. Для этого модель обращается к составленному заранее словарю токенов. На выходе для каждого текста образуются векторы заданной длины.

In [6]:
device = torch.device('cuda')

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [7]:
%%time

device = torch.device('cuda')

bert_model = transformers.BertModel.from_pretrained('bert-base-uncased')
bert_model.to(device)

batch_size = 50
embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = bert_model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

train_x = np.concatenate(embeddings)
train_y = train['toxic']

100%|██████████| 400/400 [04:02<00:00,  1.65it/s]

Wall time: 4min 8s


In [8]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.5 GB
Cached:    2.1 GB


Проделываем все тоже самое с тестовой выборкой

In [9]:
%%time
# tockenize
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-cased')
tokenized = test['text'].apply(
    lambda x: tokenizer.encode(x[:128], add_special_tokens=True))

# count max qty of tockens
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
# add 0 to strokes which lenght less than max
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
# make a mask to highlight important tockens
attention_mask = np.where(padded != 0, 1, 0)
padded.shape

Wall time: 3.96 s


(10000, 127)

In [12]:
%%time
device = torch.device('cuda')

bert_model = transformers.BertModel.from_pretrained('bert-base-uncased')
bert_model.to(device)

batch_size = 50
embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = bert_model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

test_x = np.concatenate(embeddings)
test_y = test['toxic']

100%|██████████| 200/200 [00:27<00:00,  7.14it/s]

Wall time: 30.4 s


<a id='b_2'></a>
### 2. Обучение

8. На финальном этапе модели передаем признаки (векторы). И она прогнозирует эмоциональную окраску текста — 0 («отрицательная») или 1 («положительная»).

In [13]:
lr_model = LogisticRegression(random_state=333, class_weight='balanced', solver='liblinear')
lr_model.fit(train_x, train_y)

train_pred = lr_model.predict(train_x)
test_pred = lr_model.predict(test_x)

print('f1 на трейне:', f1_score(train_y, train_pred))
print('f1 на тесте:', f1_score(test_y, test_pred))

f1 на трейне: 0.7639659825838977
f1 на тесте: 0.3242313480971834


In [14]:
sgd_model = SGDClassifier(random_state=333, class_weight='balanced')
sgd_model.fit(train_x, train_y)

train_pred = sgd_model.predict(train_x)
test_pred = sgd_model.predict(test_x)

print('f1 на трейне:', f1_score(train_y, train_pred))
print('f1 на тесте:', f1_score(test_y, test_pred))

f1 на трейне: 0.7255681392034801
f1 на тесте: 0.2127566622979467


На тесте очень слабый показатель, возможно ситуация будет лучше если увеличить выборку и убрать downsampling

<a id='t'></a>
# tf-idf

<a id='t_1'></a>
### 1. Подготовка

In [15]:
df = df.sample(80000).reset_index(drop=True)

1. Слова лемматризируем
2. Текст очищаем от стоп-слов и ненужных символов

In [16]:
def stemmatize(text):
    stemmer = nltk.stem.snowball.SnowballStemmer('english')
    
    words = text.split()
    stemm_list = []
    for word in words:
        stemm_list.append(stemmer.stem(word))
        
    return ' '.join(stemm_list)


def clear_text(text):
    processed_text = re.sub(r'[^a-zA-Z]', ' ', text)
    processed_text = ' '.join(processed_text.split())
    
    return processed_text

In [17]:
%%time
df['stem'] = df['text'].apply(lambda x: stemmatize(clear_text(x)))

Wall time: 49.1 s


Выделим тестовую выборку

In [18]:
train, test = train_test_split(df, test_size=0.2, random_state=333)

train_x = train.drop('toxic', axis=1)
train_y = train['toxic']

test_x = test.drop('toxic', axis=1)
test_y = test['toxic']

print('Размер трейна', train_x.shape)
print('Размер теста', test_x.shape)

Размер трейна (64000, 2)
Размер теста (16000, 2)


3. Готовим векторы признаков

In [19]:
%%time
count_tfidf = TfidfVectorizer(stop_words='english')

tf_idf = count_tfidf.fit_transform(np.array(train_x['stem']))
train_x = pd.DataFrame(tf_idf.toarray())

tf_idf = count_tfidf.transform(np.array(test_x['stem']))
test_x = pd.DataFrame(tf_idf.toarray())

print('Размер трейна', train_x.shape)
print('Размер теста', test_x.shape)

Размер трейна (64000, 75938)
Размер теста (16000, 75938)
Wall time: 4.39 s


<a id='t_2'></a>
### 2. Обучение

LogisticRegression

In [20]:
%%time
lr_model = LogisticRegression(random_state=333, class_weight='balanced', solver='liblinear')
lr_model.fit(train_x, train_y)

train_pred = lr_model.predict(train_x)
test_pred = lr_model.predict(test_x)

print('f1 на трейне:', f1_score(train_y, train_pred))
print('f1 на тесте:', f1_score(test_y, test_pred))

f1 на трейне: 0.8447667842171802
f1 на тесте: 0.7448503158472948
Wall time: 9min 59s


In [ ]:
%%time
sgd_model = SGDClassifier(random_state=333, class_weight='balanced')
sgd_model.fit(train_x, train_y)

train_pred = sgd_model.predict(train_x)
test_pred = sgd_model.predict(test_x)

print('f1 на трейне:', f1_score(train_y, train_pred))
print('f1 на тесте:', f1_score(test_y, test_pred))

<a id='c'></a>
# Выводы

Балансировку классов провели с помощью внутринних механизмов моделей.

Лучший результат f1 0.755 на тесте показала логистическая регрессия поверх tf-idf. Возможно лог регрессия поверх Bertа показала бы результат лучше при большей выборке.